In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import os

In [9]:
dirpath = os.getcwd()
videopath = os.path.join(dirpath, "..", "..", "udder_video", "video_metadata")
video =  pd.read_csv(os.path.join(videopath, "video_metadata_20231117.csv"))
data_path = os.path.join(dirpath, "..", "data")

In [10]:
milk_df= pd.read_csv(os.path.join(data_path, "delpro_visit_lactation.csv")) # , header = 0,  names = names, dtype = dtypes)
milk_df = milk_df[milk_df['date'].notna()]
# date formats: date, begin time, end time
milk_df.date = [datetime.strptime(day, "%Y-%m-%d") for day in milk_df.date]
milk_df.begin_time = [datetime.strptime(day, "%m/%d/%Y %I:%M %p") for day in milk_df.begin_time]
# milk report time is in AM -PM format
milk_df["start_hour"]=[date.hour for date in milk_df.begin_time]
milk_df["day"]=[date.day for date in milk_df.begin_time]
milk_df["month"]=[date.month for date in milk_df.begin_time]
milk_df["year"]=[date.year for date in milk_df.begin_time]
milk_df = milk_df[(milk_df.day == 17) & (milk_df.month == 11) & (milk_df.year == 2023)].reset_index()
milk_df = milk_df.drop(["date","day", "year", "month"], axis = 1)

# fix time and date to make them match
# I give a +-10 min margin form video to milking
video["start_hour0"]=[datetime.strptime(date, '%H:%M:%S').hour for date in video.time]
video["start_hour1"]=[datetime.strptime(date, '%H:%M:%S') + timedelta(minutes = 10) for date in video.time]
video["start_hour1"]=[hour.hour for hour in video.start_hour1]
video["start_hour2"]=[datetime.strptime(date, '%H:%M:%S') - timedelta(minutes = 10) for date in video.time]
video["start_hour2"]=[hour.hour for hour in video.start_hour2]

milk_videos = pd.concat([pd.merge(video, milk_df,  left_on=["cow", "start_hour0"], right_on=["animal_number", "start_hour"]),  \
                         pd.merge(video, milk_df,  left_on=["cow", "start_hour1"], right_on=["animal_number", "start_hour"]), \
                         pd.merge(video, milk_df,  left_on=["cow", "start_hour2"], right_on=["animal_number", "start_hour"])],\
                        axis= 0, ignore_index = True).drop_duplicates().reset_index(drop = True)

milk_videos = milk_videos.drop(["start_hour", "start_hour0", "start_hour1", "start_hour2"], axis = 1)
milk_videos.to_csv(os.path.join(data_path, "milk_videos_visit.csv"), index = False)

In [16]:
lac_df= pd.read_csv(os.path.join(data_path, "merged2_lactation.csv"))
lac_videos = pd.merge(video, lac_df, how = "left",  left_on="cow", right_on="animal_number")
lac_videos = lac_videos.drop(["start_hour0", "start_hour1", "start_hour2"], axis = 1)
lac_videos.to_csv(os.path.join(data_path, "merged2_videos_lactation.csv"), index = False)